# Mid-Atlantic Beer Data

## Data Gathering

#### The Universe of Beer Reviews and Beer-Related Databases

There is a bevy of beer review data on the Internet. Some of the main beer review website/applications include: 
* [BeerAdvocate](https://www.beeradvocate.com) 
* [RateBeer](https://www.ratebeer.com)
* [Untappd](https://untappd.com/home)

Additionally, there are more specialized Beer-focused publications that include reviews, typically with a more limited selection of esoteric or limited-release beers:
* [The Beer Connoiseur](https://beerconnoisseur.com)
* [Craft Beer and Brewing](https://beerandbrewing.com)
* [The Full Pint](https://thefullpint.com/beer-reviews/)
* [All About Beer](http://allaboutbeer.com)
* [Draft Magazine](https://draftmag.com)

General brewery databases:
* [Open Brewery DB](https://www.openbrewerydb.org)
* [BreweryDB](https://www.brewerydb.com/developers)

Both [RateBeer](https://www.ratebeer.com/api-documentation.asp) and [Untappd](https://untappd.com/api/register#) have developer APIs, but both require explicit approval for API keys. Untappd in particular places restrictions on API access for pure research and analytics purposes. Given the turnaround time for API key access, I'm relying heavily on web scraping, which is not explicitly prohibited per each website's Robots Exclusion Protocol page. I have avoided scraping BeerAdvocate given reports of the owner's proclivity towards legal action against prior web scrapers.

In addition to scraping beer review data, I've also incorporated data from the very cool beer databases [Open Brewery DB](https://www.openbrewerydb.org) and [BreweryDB](https://www.brewerydb.com).

#### Beer Style Guidelines

* [BJCP](https://www.bjcp.org/docs/2015_Guidelines_Beer.pdf)
* [Brewers Association](https://www.brewersassociation.org/press-releases/brewers-association-releases-2019-beer-style-guidelines/)

I also consulted the leading beer style guidelines. At first, I considered maybe this could be a source of features for the model, but given the diversity of beers even within the same style category, the style guides themselves were not helpful except for aiding in manual mapping of beers to broader categories.

## Data Processing and Exploration (Aggregate Beer Ratings)

I created a web-scraping script which scrapes all the beers and its associated characteristics (name, style, ABV, aggregate RateBeer rating, date added to RateBeer) from each brewery in a given url associated with a specific region. For this analysis I'm restricting my scope to breweries within Delaware, Maryland, Virginia, Washington D.C., and West Virginia.

The web scraping script outputted separate csv files for each state composed of all beers from breweries in that state.

In [1]:
# Import pandas, numpy for data cleaning and viz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline

pd.options.display.max_columns = 5_000 # So I can see all the columns rather than just ...

### Reading and Cleaning Beer Listing Data from RateBeer

In [2]:
# Reading in and combining .CSVs of scraping
dc = pd.read_csv('../data_gathering/scrape_output/brewlist_DC.csv')
md = pd.read_csv('../data_gathering/scrape_output/brewlist_MD.csv')
va = pd.read_csv('../data_gathering/scrape_output/brewlist_VA.csv')

dmv_beers = pd.concat([dc, md, va], ignore_index=True, sort=False)

In [3]:
dmv_beers.head()

,abv,address,brewery,country,date_added,google_map_link,link,name,num_ratings,postal_code,rating,state,style,style_rating,town
0,0.083,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,6/3/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-ultrafresh/423469/,3 Stars #ultrafresh,14,20011,3.47,Washington DC,IIPA - Imperial / Double IPA,41.0,"Washington, DC"
1,0.067,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,10/6/2018,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-bout-that-life/6...,3 Stars 'Bout That Life,3,20011,3.18,Washington DC,Sour / Wild Beer,NaN,"Washington, DC"
2,0.055,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,4/17/2017,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-aslin-flip-the-s...,3 Stars / Aslin Flip the Script,5,20011,3.20,Washington DC,Sour / Wild Beer,NaN,"Washington, DC"
3,0.072,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,5/13/2018,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-captain-lawrence...,3 Stars / Captain Lawrence 77's and Bonnevilles,3,20011,3.21,Washington DC,IIPA - Imperial / Double IPA,NaN,"Washington, DC"
4,0.065,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,9/18/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-charm-city-two-h...,3 Stars / Charm City Two Headed Unicorn,9,20011,3.44,Washington DC,Sour / Wild Beer,41.0,"Washington, DC"


In [4]:
dmv_beers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18294 entries, 0 to 18293
Data columns (total 15 columns):
abv                17109 non-null float64
address            18073 non-null object
brewery            18294 non-null object
country            18294 non-null object
date_added         18294 non-null object
google_map_link    18294 non-null object
link               18294 non-null object
name               18294 non-null object
num_ratings        18294 non-null int64
postal_code        18240 non-null object
rating             13494 non-null float64
state              18294 non-null object
style              18294 non-null object
style_rating       2582 non-null float64
town               18292 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 2.1+ MB


In [5]:
# Dropping sytle rating column and dropping remaining nulls (that is beers without ratings)
dmv_beers.drop(columns=['style_rating'], inplace=True)
dmv_beers.dropna(inplace=True)

In [6]:
dmv_beers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12601 entries, 0 to 18292
Data columns (total 14 columns):
abv                12601 non-null float64
address            12601 non-null object
brewery            12601 non-null object
country            12601 non-null object
date_added         12601 non-null object
google_map_link    12601 non-null object
link               12601 non-null object
name               12601 non-null object
num_ratings        12601 non-null int64
postal_code        12601 non-null object
rating             12601 non-null float64
state              12601 non-null object
style              12601 non-null object
town               12601 non-null object
dtypes: float64(2), int64(1), object(11)
memory usage: 1.4+ MB


### Data Cleaning

In [7]:
def strip_col(df, col_list):
    """
    Trim whitepace on specified columns of object dtype.
    
    Parameters
    ----------
    df : pandas dataframe
    col_list : list of specified name of columns (str)

    Returns
    -------
    revised pandas dataframe

    """
    for col in col_list:
        if df[col].dtype == 'O':
            df[col] = df[col].map(lambda x: x.strip())
        else:
            print("The specified column is not an object (string) datatype.")
    return df

In [8]:
# Some columns may have extra white space - let's clean it
strip_col(dmv_beers, col_list=['brewery', 'name', 'style', 'address','town']);

Thanks to [Jason Ong (jasonong) on Github](https://github.com/jasonong/List-of-US-States) for saving me some time from manually creating the state to abbreviation mapping.

In [9]:
stateabbrev_df = pd.read_csv('./state_abbreviations.csv')
stateabbrev_dict = stateabbrev_df.set_index('State').to_dict()['Abbreviation']

In [10]:
def replace_via_dict(df, col, abbrev_dict):
    df[col].replace(abbrev_dict, inplace=True)
    return df

In [11]:
# Ratebeer labels states oddly - DC needs to be corrected
dmv_beers['state'].value_counts()

Virginia         8002
Maryland         3552
Washington DC    1047
Name: state, dtype: int64

In [12]:
dmv_beers['state'].replace({'Washington DC':'District of Columbia'}, inplace=True)

In [13]:
# Reassign state column to abbreviations
dmv_beers = replace_via_dict(dmv_beers, 'state', stateabbrev_dict)

In [14]:
# All the items in country column should be 'USA'
dmv_beers['country'].value_counts()

USA    12601
Name: country, dtype: int64

In [15]:
# There are a few variations on Washington DC
dmv_beers[dmv_beers['town'].str.contains('Washington')]['town'].value_counts()

Washington        739
Washington DC     160
Washington, DC    148
Name: town, dtype: int64

In [16]:
# Transforming all Washington DC variations to Washington
dmv_beers['town'].replace({'Washington DC':'Washington','Washington, DC':'Washington'}, inplace=True)
dmv_beers[dmv_beers['town'].str.contains('Washington')]['town'].value_counts()

Washington    1047
Name: town, dtype: int64

In [17]:
# Breakout of beer count
dmv_beers['state'].value_counts()

VA    8002
MD    3552
DC    1047
Name: state, dtype: int64

There are some geographical attributes of the breweries that may be helpful in filtering the recommendations but are not useful features in the recommender model itself. I'm going to create a new dictionary with that information to be utilized later and drop those columns from the beer list dataframe.

In [18]:
dmv_beer_to_location = dmv_beers[['name','brewery','address','town','state','postal_code']].set_index('name').T.to_dict()

/Users/willkarnasiewicz/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [19]:
features = ['name','abv', 'date_added', 'num_ratings','rating','style']
dmv_beer_data = dmv_beers[features]
dmv_beer_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12601 entries, 0 to 18292
Data columns (total 6 columns):
name           12601 non-null object
abv            12601 non-null float64
date_added     12601 non-null object
num_ratings    12601 non-null int64
rating         12601 non-null float64
style          12601 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 689.1+ KB


In [20]:
dmv_beer_data.head()

,name,abv,date_added,num_ratings,rating,style
0,3 Stars #ultrafresh,0.083,6/3/2016,14,3.47,IIPA - Imperial / Double IPA
1,3 Stars 'Bout That Life,0.067,10/6/2018,3,3.18,Sour / Wild Beer
2,3 Stars / Aslin Flip the Script,0.055,4/17/2017,5,3.20,Sour / Wild Beer
3,3 Stars / Captain Lawrence 77's and Bonnevilles,0.072,5/13/2018,3,3.21,IIPA - Imperial / Double IPA
4,3 Stars / Charm City Two Headed Unicorn,0.065,9/18/2016,9,3.44,Sour / Wild Beer


In [21]:
dmv_beer_data['year_added'] = dmv_beer_data['date_added'].map(lambda x: str(x)[-4:])

/Users/willkarnasiewicz/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
dmv_beer_data.head()

,name,abv,date_added,num_ratings,rating,style,year_added
0,3 Stars #ultrafresh,0.083,6/3/2016,14,3.47,IIPA - Imperial / Double IPA,2016
1,3 Stars 'Bout That Life,0.067,10/6/2018,3,3.18,Sour / Wild Beer,2018
2,3 Stars / Aslin Flip the Script,0.055,4/17/2017,5,3.20,Sour / Wild Beer,2017
3,3 Stars / Captain Lawrence 77's and Bonnevilles,0.072,5/13/2018,3,3.21,IIPA - Imperial / Double IPA,2018
4,3 Stars / Charm City Two Headed Unicorn,0.065,9/18/2016,9,3.44,Sour / Wild Beer,2016


In [23]:
dmv_beer_data.drop(columns=['date_added'], inplace=True)

/Users/willkarnasiewicz/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
dmv_beer_data.head()

,name,abv,num_ratings,rating,style,year_added
0,3 Stars #ultrafresh,0.083,14,3.47,IIPA - Imperial / Double IPA,2016
1,3 Stars 'Bout That Life,0.067,3,3.18,Sour / Wild Beer,2018
2,3 Stars / Aslin Flip the Script,0.055,5,3.20,Sour / Wild Beer,2017
3,3 Stars / Captain Lawrence 77's and Bonnevilles,0.072,3,3.21,IIPA - Imperial / Double IPA,2018
4,3 Stars / Charm City Two Headed Unicorn,0.065,9,3.44,Sour / Wild Beer,2016


In [25]:
dmv_beer_data_dummied = pd.get_dummies(dmv_beer_data, columns=['style','year_added'])

In [26]:
dmv_beer_data_dummied.set_index('name', inplace=True)

In [27]:
dmv_beer_data_dummied.shape

(12601, 134)

### Exporting Data for Use in Recommender System and Further Analysis

#### List of Beers in System:

In [28]:
beer_list = [_ for _ in dmv_beer_data_dummied.index]

with open('../data/beerlist.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(beer_list, f, pickle.HIGHEST_PROTOCOL)

#### Sparse Matrix of Beer Details for Basic Content Recommendation

In [29]:
dmv_content_dummy_sparse = sparse.csr_matrix(dmv_beer_data_dummied)

In [30]:
sparse.save_npz('../data/dmv_beer_content_sparse',dmv_content_dummy_sparse)

#### DMV Beer dataframe

In [31]:
dmv_beers.to_csv('../data/dmv_beer_full_database.csv',index=False)

#### DC Beer to Location dictionary

In [32]:
with open('../data/beer_to_location.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dmv_beer_to_location, f, pickle.HIGHEST_PROTOCOL)

#### DMV Beer Link dataframe

In [33]:
dmv_beers[['name','link']].to_csv('../data/dmv_beerlinks.csv')